# Estimación de máximo estiimador de verosimilitud del parámetro $\lambda$ de una distribución logarítmica

Para la solución del problema se hace uso del método de Newton-Raphson, cuyo implementación se muestra a continuación.

## Líbrerias y clases que se usaran

In [1]:
import math
import sympy
import numpy as np
import plotly.graph_objects as go

class RealFunction:

    def __call__(self, x):
        a = self._evaluate(x)
        return a

    def __init__(self,  expression = "", function = None):
        self._my_symbols = {'x': sympy.Symbol('x', real=True)}
        if expression != "":
            self._expression = expression
            self._my_func = sympy.parsing.sympy_parser.parse_expr(expression, self._my_symbols)
        else:
            self._my_func = function
        self._evaluate = sympy.utilities.lambdify(self._my_symbols['x'], self._my_func, modules=["numpy"])
        self._der = dict()
    
    def der(self, ord):
        if(ord not in self._der):
            _derk = self._my_func
            for _ in range(ord):
                _derk = sympy.diff(_derk, self._my_symbols['x'])
            self._der[ord] = self.__class__(function = _derk)
        return self._der[ord]

## Algoritmo Newton-Raphson

La función siguiente función emplea el método Newton-Rapshon para calcular las raices de una función $f$. Como parametros necesito la función $f$, el punto inicial $x0$, la cantidad de iteraciones máximas $iterMax$, la tolerancia $tol$, y opcionalmente el minimo y máximo valor donde buscar como $tmin$ y $tmax$ respectivamente.

In [2]:
def NewtonRaphson(x0, f, iterMax, tol, tmin = None, tmax = None):
    xk  = x0
    res = 0
    for k in range(iterMax):
        fk = float(f(xk))
        if fk<tol and fk>-tol:
            res = 1
            break
        else:
            dfk = float(f.der(1)(xk))
            if dfk!=0:
                xk = xk - fk/dfk
                if(tmin != None):
                    xk = max(xk, tmin)
                if(tmax != None):
                    xk = min(xk, tmax)
            else:
                res = -1
                break
    return xk,k,res

Impresión de resultados del algoritmo Newton-Raphson

In [3]:
def interpretador_newton(f, x0, iterMax, tol):
    xk,k,res = NewtonRaphson(x0 = x0, iterMax = iterMax, f = f, tol = tol)
    
    print("Para la funcion real f(x) = {}\nCon punto inicial x0 = {}, donde f(x0) = {}".format(f._my_func, x0, f(x0)))

    print("El metodo de Newton, obtuvo res = {}, es decir:\n".format(res))
    if res==1:
        print('\tEncontro una raiz cerca de xk = {}\n\tdonde f(xk) = {},\n\ten {} iteraciones'.format(xk,f(xk), k))
    elif res==0:
        print('\tNo converje.')
        return
    else:
        print('\tSe cancelo la derivada en algun momento')
        return
    print("")

    return xk, res

## Frecuencias Esperedas

Recibe como parámetros las frecuencias observadas, así como el $\hat{\lambda}$ de una distirbución logarítmica. Imprime una tabla de las frecuencias observadas y esperadas.

In [4]:
def exp_frec_log_dist(frec, emv):
    n = sum(frec[1])
    print("Las frecuencias esperados son:")
    print("No. defectuosos", frec[0])
    print("frec. obs.", frec[1])
    print("frec. esp.", np.concatenate((np.round([(1-math.exp(-emv))**(j+1)/((j+1)*emv)*n for j in range(4)], 1), np.round([(1-sum((1-math.exp(-emv))**(j+1)/((j+1)*emv) for j in range(4)))*n],1))))
    print("")

## Intervalo de verosimilitud

Recibe como parámetros una función de verosimilitud $L$, el porcentaje $p$, el $emv$ de $L$, la cantidad de pasos para calcularlo $iterMax$ y la tolerancia de error $tol$. Imprime el intervalo de verosimilitud del $p\%$.

In [5]:
def likelihood_interval_mountain(L, p, emv, iterMax, tol):
    
    g = RealFunction(function = L._my_func - L(emv) - math.log(p))
    xk,k,res = NewtonRaphson(emv + 0.001, g, iterMax, tol, tmin = emv, tmax = 1)
    R_lambda_tmax = xk
    xk,k,res = NewtonRaphson(emv - 0.001, g, iterMax, tol, tmin = 0.000001, tmax = emv)
    R_lambda_tmin = xk
    print("El intervalo de verosimilitud del {}% es: [{},{}]".format(p*100, R_lambda_tmin, R_lambda_tmax))

## Código Prinicipal

Declaración de parametros y funciones para estimar $\hat{\lambda}$, frecuencias esperadas e intervalo de verosimilitud $p$.

In [6]:
def c_1(frec):
    return sum((frec[0][j]+1)*frec[1][j] for j in range(len(frec[0])))
def c_2(frec):
    return sum(frec[1][j] for j in range(len(frec[0])))
    
def main():
    
    frec = np.array([[0,1,2,3,4],[147,36,13,4,0]])
    f = RealFunction(expression = "{}".format(c_1(frec)) + "/(exp(x)-1) - " + "{}".format(c_2(frec))+"/x")
    iterMax = int(1e6)
    tol = 1e-5
    x0 = 0.1
    emv_lambda, res = interpretador_newton(f, x0, iterMax, tol)
    
    print("La función de Información de Lambda en su emv es:", f.der(1)(emv_lambda),"\n")

    exp_frec_log_dist(frec, emv_lambda)

    f = RealFunction(expression = "{}".format(c_1(frec)) + "*log(1-exp(-x)) - " + "{}".format(c_2(frec)) + "*log(x)")
    likelihood_interval_mountain(f, 0.1, emv_lambda, iterMax, tol)
    

In [7]:
main()

Para la funcion real f(x) = 274/(exp(x) - 1) - 200/x
Con punto inicial x0 = 0.1, donde f(x0) = 605.2829528683615
El metodo de Newton, obtuvo res = 1, es decir:

	Encontro una raiz cerca de xk = 0.5997370966729684
	donde f(xk) = 7.350172381848097e-08,
	en 7 iteraciones

La función de Información de Lambda en su emv es: -183.30732984343024 

Las frecuencias esperados son:
No. defectuosos [0 1 2 3 4]
frec. obs. [147  36  13   4   0]
frec. esp. [150.4  33.9  10.2   3.5   2. ]

El intervalo de verosimilitud del 10.0% es: [0.4561586241961348,0.7746682846541133]
